# Project 3

In this project, you will perform a logistic regression on admissions data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna()
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 0 to 399
Data columns (total 4 columns):
admit       397 non-null int64
gre         397 non-null float64
gpa         397 non-null float64
prestige    397 non-null float64
dtypes: float64(3), int64(1)
memory usage: 15.5 KB


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables.  Look at the documentation for pd.crosstab

In [4]:
prestige = pd.crosstab(index=df['prestige'], columns='count')
prestige
#pd.crosstab(index=df['prestige'], columns='count').sum()

col_0,count
prestige,
1.0,61
2.0,148
3.0,121
4.0,67


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [5]:
prestige_dummies = pd.get_dummies(df['prestige'])
prestige_dummies.head()

,1.0,2.0,3.0,4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


In [6]:
prestige_dummies.rename(columns={1.0: 'Prestige1', 2.0: 'Prestige2', 3.0: 'Prestige3', 4.0: 'Prestige4'}, inplace=True)
prestige_dummies.head()

,Prestige1,Prestige2,Prestige3,Prestige4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3 (always 1 less than the number of calss variables) 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [7]:
cols_to_keep = ['admit', 'gre', 'gpa']
handcalc = df[cols_to_keep].join(prestige_dummies)
handcalc.head()

,admit,gre,gpa,Prestige1,Prestige2,Prestige3,Prestige4
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [8]:
len(handcalc['admit'])

397

In [9]:
len(handcalc[handcalc['admit']==0])

271

In [10]:
handcalc['admit'].sum()

126

In [11]:
len(handcalc[handcalc['Prestige1']==0])

336

In [12]:
handcalc['Prestige1'].sum()

61

In [13]:
prestige = pd.crosstab(index=handcalc['Prestige1'], columns="count")
prestige

col_0,count
Prestige1,
0,336
1,61


In [21]:
handcalc['Prestige4'].value_counts()

0    330
1     67
Name: Prestige4, dtype: int64

In [20]:
comb = pd.crosstab(index=handcalc['admit'], columns=[handcalc['Prestige1'],handcalc['Prestige2'],handcalc['Prestige3'],handcalc['Prestige4']], margins=True)
comb.columns = ['prestige 1','prestige 2','prestige 3','prestige 4', 'rowtotal']
comb.index = ['admit 0','admit 1', 'coltotal']
comb

,prestige 1,prestige 2,prestige 3,prestige 4,rowtotal
admit 0,55,93,95,28,271
admit 1,12,28,53,33,126
coltotal,67,121,148,61,397


In [16]:
comb = pd.crosstab(index=df['admit'], columns=df['prestige'])
comb.columns = ['prestige 1','prestige 2','prestige 3','prestige 4']
comb.index = ['admit 0','admit 1']
comb

,prestige 1,prestige 2,prestige 3,prestige 4
admit 1,28,95,93,55
admit 0,33,53,28,12


In [23]:
pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


In [24]:
handcalc

,admit,gre,gpa,Prestige1,Prestige2,Prestige3,Prestige4
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1
5,1,760.0,3.00,0,1,0,0
6,1,560.0,2.98,1,0,0,0
7,0,400.0,3.08,0,1,0,0
8,1,540.0,3.39,0,0,1,0
9,0,700.0,3.92,0,1,0,0


In [25]:
prestige_1 = pd.crosstab(index=handcalc['Prestige1'], columns='count')
prestige_1

col_0,count
Prestige1,
0,336
1,61


In [26]:
prestige_1 = pd.crosstab(index=handcalc['admit'], columns='count')
prestige_1

col_0,count
admit,
0,271
1,126


In [27]:
# crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
comb1 = pd.crosstab(index=handcalc['admit'], columns=handcalc['Prestige1'])
comb1.columns = ['prestige 1', 'not prestige 1']
comb1.index = ['admit 0','admit 1']
comb1

,prestige 1,not prestige 1
admit 0,243,28
admit 1,93,33


In [28]:
# crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
comb2 = pd.crosstab(index=handcalc['Prestige1'], columns=handcalc['admit'])
comb2.columns = ['admit 0','admit 1']
comb2.index = ['prestige 1', 'not prestige 1']
comb2

,admit 0,admit 1
prestige 1,243,93
not prestige 1,28,33


In [29]:
comb3 = pd.crosstab(handcalc['admit'], handcalc['Prestige1'], rownames=['admit'], colnames=['Prestige1'])
comb3

Prestige1,0,1
admit,,
0,243,28
1,93,33


In [30]:
comb4 = pd.crosstab(handcalc['Prestige1'], handcalc['admit'], rownames=['Prestige1'], colnames=['admin'])
comb4

admin,0,1
Prestige1,,
0,243,93
1,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [31]:
comb4.iloc[1][1] / (comb4.iloc[1].sum() - comb4.iloc[1][1])

1.1785714285714286

odds ratio:  33:28

In [39]:
comb1

,prestige 1,not prestige 1
admit 0,243,28
admit 1,93,33


In [32]:
comb1['prestige 1'] #why is this flipped?

admit 0    243
admit 1     93
Name: prestige 1, dtype: int64

In [33]:
comb1['not prestige 1'] #why is this flipped?

admit 0    28
admit 1    33
Name: not prestige 1, dtype: int64

In [34]:
comb1['prestige 1']['admit 1'==1].sum() #why is this flipped?

243

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [35]:
comb4

admin,0,1
Prestige1,,
0,243,93
1,28,33


In [36]:
comb4.iloc[0][1] #why is this the same?

93

In [37]:
comb4.loc[0][1] #why is this the same?

93

In [38]:
comb4.loc[0][1] / ((comb4.loc[0][1] + comb4.loc[1][1]) - comb4.loc[0][1])

2.8181818181818183

#### 3.3 Calculate the odds ratio

odds ratio:  93:33

#### 3.4 Write this finding in a sentenance: 

Answer: 

#### 3.5 Print the cross tab for prestige_4

#### 3.6 Calculate the Odds Ratio 

#### 3.7 Write this finding in a sentence

Answer:

## Part 4. Analysis

In [12]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']

## Dropping one of the dummy columns
data = df[cols_to_keep].join(prestige_dummies.iloc[:, 1:])
data.head()

,admit,gre,gpa,2.0,3.0,4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


#### 4.1 Create the X and Y variables

#### 4.2 Fit the model - 

 - Load sklearn's logistic regression
 - Create the regression object
 - Fit the model

#### 4.3 Print the coefficients

#### 4.4 Calculate the odds ratios of the coeffiencents

hint 1: np.exp(X)

#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.